In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#IMPORTANT: Set folder_path to current location of 'shared task' folder, then run all.

save: saves intermediate data

skip_gated_model: skips step which requires gated model access. No data will be lost. (meta-llama/Llama-3.2-3B-Instruct)

In [4]:
folder = r'/content/drive/MyDrive/digital text analysis/NLP/shared task'
save = False
skip_gated_model = True

### for running notebooks without overloading memory

In [6]:
def run_notebook(sub_path, keep_var):
  init_var = set(globals().keys())
  loc = f'{folder}/{sub_path}'
  %run "{loc}"
  cur_var = set(globals().keys())
  new_var = cur_var - init_var
  print('Deleting unnecessary variables...')
  for var in new_var:
    if var not in keep_var and var in globals():
      del globals()[var]

##Statistics:

In [ ]:
run_notebook(sub_path='subscripts/train/stats.ipynb',keep_var={'authors'}) #pass add graphs, label imbalance

##Preprocess:

**Langdetect**:

Preparatory cleaning

Runs all comments through langdetect to exclude any that are not English or without features.

This does take a while.

In [ ]:
run_notebook(sub_path='subscripts/train/langdetect.ipynb',keep_var={'authors'})

**Humility scoring**: few-shot prompting

I used a gated model for this, so your hf_token will need access: meta-llama/Llama-3.2-3B-Instruct

If skipped data will be loaded from list.

In [7]:
run_notebook(sub_path='subscripts/train/humility.ipynb',keep_var={'full_train_data','test_data'})

NB loaded in var
Skipping model and using local data
Loaded data from list
Preparing data for training...

Fun fact! Someone quoted Moby Dick all over the validation set! ^^
'Latin from the books of the Laws of England, which taken along with the context, means, that of all whales captured by anybody on the coast of that land, the King, as Honorary Grand Harpooneer, must have the head, and the Queen be respectfully presented with the tail. A division which, in the whale, is much like halving an apple; there is no intermediate remainder. Now as this law, under a modified form, is to this day in force in England; and as it offers in various respects a strange anomaly touching the general law of Fast and Loose-Fish, it is here treated of in a separate chapter, on the same courteous principle that prompts the English railways to be at the expense of a separate car, specially reserved for the accommodation of royalty. In the first place, in curious proof of the fact that the above-mentioned

**Question placement scoring** (not implemented)

Uses a zero-shot multi-label classification pipeline with *facebook/bart-large-mnli* to score comments for their adherence to the Pera questions (Q1 / Q2 / Q3) for meaningful concatenation.

Surprisingly the most effective labels on the validation set were the literal questions from the Pera interview. 'an answer to' is added since the model is prompted 'this is a text about... [label]' to assign a score.


```
labels = [
    'an answer to: "Please describe a situation where you were presented with a problem outside of your comfort zone and where you were able to come up with a creative solution."',
    'an answer to: "Tell us about a time when you have failed or made a mistake. What happened? What did you learn from this experience?"',
    'an answer to: "Describe a situation in which you got a group of people to work together as a team. Did you encounter any issues? What was the end result?"'
]
```

In [ ]:
#run_notebook(sub_path='subscripts/train/Q_placement_scoring.ipynb',keep_var={pass})

Other numerical features (not implemented)

In [ ]:
#run_notebook(sub_path=)

### Train model

Bert with multiple classification heads

In [ ]:
train_mode = True
run_notebook(sub_path='subscripts/train/bert_multi_classhead.ipynb',keep_var={})

Using device: cpu
No 'numerical_features' key found in the first sample of full_train_data, or it's None. Assuming 0 numerical features.
Sample full_train_data point (user_id: N/A, num_comments: 5)
Distribution of composite labels for traits ['extraversion', 'openness']:
Counter({np.str_('2-2'): 615, np.str_('0-2'): 601, np.str_('0-0'): 200, np.str_('2-0'): 80})

Successfully performed stratified split on full_train_data.
Size of data for grid search training: 1196
Size of data for grid search validation: 300

Starting Grid Search. Total combinations: 16

GRID SEARCH TRIAL 1/16

--- Starting Trial with Params: {'learning_rate': 2e-05, 'dropout_rate': 0.1, 'batch_size': 16, 'attention_hidden_dim': 64, 'n_comments_to_process': 3, 'num_epochs_trial': 5} ---


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

  Epoch 1/5
